In [ ]:
import sqlite3
import pandas as pd
import logging
from contextlib import contextmanager
from dataclasses import dataclasses 


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S', 
    handlers=[
        logging.FileHandler("score_rh.log"),  # Salva logs em um arquivo
        logging.StreamHandler()  # Exibe logs no console
    ]
)

DB_PATH = "/app/data/teste___desafio_técnico_-_analista_de_dados_pleno_anexo_db.db"
SQL_SCRIPT_PATH = "/app/scripts/canditados_pre_selecionados.sql"
OUTPUT_CSV = "/app/output/candidatos_pre_selecionados.csv"

# Conexao DB

In [19]:
@contextmanager
def conectar_banco():
    """Gerenciador de contexto para conexão com o banco de dados."""
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH)
        yield conn  # Retorna a conexão para ser usada no bloco `with`
    except Exception as e:
        logging.error(f"Erro ao conectar ao banco de dados: {e}")
        raise
    finally:
        if conn:
            conn.close()  # Fecha a conexão após o uso

# DATA PREP

In [ ]:
def listar_tabelas():
    """Lista todas as tabelas do banco SQLite."""
    tabelas = []
    try:
        with conectar_banco() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tabelas = cursor.fetchall()
        return [t[0] for t in tabelas]
    except Exception as e:
        logging.error(f"Erro ao listar tabelas: {e}")
        return []

def visualizar_tabela(nome_tabela, haslimit=True, limite=5):
    try:
        with conectar_banco() as conn:
            if haslimit:
                query = f"SELECT * FROM {nome_tabela} LIMIT {limite};"
            else:
                query = f"SELECT * FROM {nome_tabela};"
            df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        logging.error(f"Erro ao visualizar tabela {nome_tabela}: {e}")
        return pd.DataFrame()

def verificar_nulos():
    """Identifica colunas com valores nulos em todas as tabelas."""
    tabelas = listar_tabelas()
    for tabela in tabelas:
        df = visualizar_tabela(tabela, haslimit=False)
        
        null_counts = df.isnull().sum()
        # Filtra apenas as colunas com valores nulos
        colunas_com_nulos = null_counts[null_counts > 0]
        
        if not colunas_com_nulos.empty:
            logging.warning(f"Na tabela {tabela}, as seguintes colunas têm valores nulos:")
            for coluna, quantidade in colunas_com_nulos.items():
                logging.warning(f"  - {coluna}: {quantidade} valores nulos")
        else:
            logging.info(f"Na tabela {tabela}, não há valores nulos.")

In [18]:
verificar_nulos()

2025-02-12 18:29:35 - INFO - Na tabela competencias, não há valores nulos.
2025-02-12 18:29:35 - INFO - Na tabela vagas, não há valores nulos.
2025-02-12 18:29:35 - INFO - Na tabela candidatos, não há valores nulos.
2025-02-12 18:29:35 - INFO - Na tabela vagacompetencia, não há valores nulos.
2025-02-12 18:29:35 - INFO - Na tabela candidato_vaga, não há valores nulos.
2025-02-12 18:29:35 - INFO - Na tabela experiencias, não há valores nulos.
2025-02-12 18:29:35 - INFO - Na tabela competencia_experiencia, não há valores nulos.


In [21]:
def executar_sqlite_query():
    try:
        with conectar_banco() as conn:
            cursor = conn.cursor()
            with open(SQL_SCRIPT_PATH, "r", encoding="utf-8") as f:
                sql_query = f.read()

            cursor.execute("PRAGMA foreign_keys = ON;") 
            df = pd.read_sql_query(sql_query, conn)

        df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        logging.info(f"Arquivo salvo com sucesso em: {OUTPUT_CSV}")

    except Exception as e:
        logging.error(f"Erro ao executar a consulta: {e}")

executar_sqlite_query()

2025-02-12 18:35:57 - INFO - Arquivo salvo com sucesso em: /app/output/candidatos_pre_selecionados.csv
